# DEMO Valider Skattemelding

Denne demoen er ment for å vise integrasjonene mot Skatteetaten sine nye validerings- og innsendingstjeneste.

In [ ]:
try: 
    import requests
    import base64
    import xmltodict
    import xml.dom.minidom
    from Steg.logge_inn_idporten import hent_idtoken, base64_decode_response, decode_dokument
    from portalmelding import valider_melding
except ImportError as e:
    try: 
        print("Mangler en avhengighet, installer dem via pip")
        !pip install python-jose[cryptography] cryptography
        !pip install xmltodict
        !pip install requests
        import xmltodict
        import requests
        from hent import hent_idtoken, base64_decode_response, decode_dokument
        from portalmelding import valider_melding
    except ImportError as err:
        print("Mangler en avhengighet, installer dem via pip")
        !python3 -m pip install 'python-jose[cryptography] cryptography'
        !python3 -m pip install xmltodict
        !python3 -m pip install requests
        import xmltodict
        import requests
        from Steg.logge_inn_idporten import hent_idtoken, base64_decode_response, decode_dokument
        from portalmelding import valider_melding


## Generer ID-porten token
Tokenet er gyldig i 300 sekunder, rekjørt denne biten om du ikke har kommet frem til validering biten før 300 sekunder 

In [ ]:
idporten_header = hent_idtoken()

### Valider tjeneste

In [ ]:
melding_xml = f"""
<?xml version="1.1" encoding="UTF-8"?>
<mvaMeldingDto xmlns="no:skatteetaten:fastsetting:avgift:mva:skattemeldingformerverdiavgift:v0.9">
<innsending>
<regnskapssystemsreferanse>3</regnskapssystemsreferanse>
<regnskapssystem>
<systemnavn>zzx</systemnavn>
<systemversjon>4.34.3</systemversjon>
</regnskapssystem>
<innsender>
<norskIdentifikator>02067623456</norskIdentifikator>
</innsender>
<innsendingTidspunkt>2022-05-26T21:35:47Z</innsendingTidspunkt>
</innsending>
<skattegrunnlagOgBeregnetSkatt>
    <skattleggingsperiode>
    <periode>
    <skattleggingsperiodeToMaaneder>mars-april</skattleggingsperiodeToMaaneder>
    </periode>
    <aar>2022</aar>
    </skattleggingsperiode>
    <fastsattMerverdiavgift>5000</fastsattMerverdiavgift>
    <mvaSpesifikasjonslinje>
<mvaKode>3</mvaKode>
<mvaKodeRegnskapsystem>25 utgående mva</mvaKodeRegnskapsystem>
<grunnlag>20000</grunnlag>
<sats>25</sats>
<merverdiavgift>5000</merverdiavgift>
</mvaSpesifikasjonslinje>
</skattegrunnlagOgBeregnetSkatt>
    <betalingsinformasjon>
<kontonummerForTilbakebetaling>
<norskKontonummer>999999999</norskKontonummer>
</kontonummerForTilbakebetaling>
<kundeIdentifikasjonsnummer>3400000932456870</kundeIdentifikasjonsnummer>
</betalingsinformasjon>
    <skattepliktig>
<organisasjonsnummer>913238254</organisasjonsnummer>
</skattepliktig>
    <meldingskategori>alminnelig</meldingskategori>
</mvaMeldingDto>
""".replace("\n","")

s = requests.Session()
s.headers = dict(idporten_header)
req_valider = valider_melding(dict(idporten_header), xml=melding_xml)
req_valider.status_code 
req_valider.content